In [ ]:
import os
from pathlib import Path

from src.services.analysis import analyze_sample
from src.services.common import logger
from src.services.constants import APPLYABLE_FILETYPES


def run_full_analysis():
    """
    Проводит полный анализ всех образцов.
    """
    db_dir = Path(os.getcwd()) / "db"          # путь к папке с БД
    input_dir = Path(os.getcwd()) / "input"    # путь к исходным данным с папками образцов
    result_dir = Path(os.getcwd()) / "result"  # путь для сохранения результатов анализа

    threads = str(os.cpu_count())  # количество потоков для параллельных вычислений (количество ядер процессора)

    samples = sorted(os.listdir(input_dir))  # список образцов в папке с входными данными

    num_samples = len(samples)  # количество образцов

    file_counter = dict()  # словарь количества файлов для анализа для каждого образца

    for sample in samples:
        files = [file for file in os.listdir(input_dir/sample) if file.split(".")[-1] in APPLYABLE_FILETYPES]
        file_counter[sample] = len(files)

    logger.info(
        "\nStart a new full analysis:\n"
        f"Number of threads: {threads}\n"
        f"Number of samples: {num_samples}\n"
        f"{samples}\n"
        f"Total number of files for the analysis: {sum(file_counter.values())}\n"
        f"Number of files for each sample: {file_counter}"
    )

    result_flags = dict()  # словарь для хранения флагов успешности анализа каждого образца

    for sample in samples:
        sample_input_dir = input_dir / sample
        sample_result_dir = result_dir / sample

        os.makedirs(sample_result_dir, exist_ok=True)

        logger.info(f"Start analyzing sample: {sample}")

        sample_analyzed = analyze_sample(
            sample=sample,
            sample_input_dir=sample_input_dir,
            sample_result_dir=sample_result_dir,
            db_dir=db_dir,
            gff_type="prodigal",
            threads=threads
        )

        result_flags[sample] = sample_analyzed

    logger.info(f"Full analysis ended successfully:\n{result_flags}")


run_full_analysis()